# 3D Graphics in JupyterLab using pythreejs: Part 2

*Yen Lee Loh (2023-5-16, 2023-6-8)*

On its own, pythreejs is inconvenient for various reasons:
- pythreejs.TorusBufferGeometry creates a torus whose symmetry axis is the z axis.
- pythreejs.CylinderBufferGeometry creates a frustum whose symmetry axis is the y axis.
- It is not easy to reorient these shapes along desired directions.
The `threed` module includes wrapper functions and utility functions to make 3D graphics easier.

## Example 1: Draw a dumbbell using `threed.sphere` and `threed.cylinder`

Make sure that `threed.py` and `monospace.png` are in the same directory as this notebook file.  Import the `threed` module:

In [1]:
import threed
import pythreejs as p3j

In [2]:
import threed; import importlib; importlib.reload (threed);  # Developer use

Create a list called `objects` and populate it.  Each object is actually a `pythreejs.Mesh`.  As a quick test, combine the objects into a `pythreejs.Scene`:

In [40]:
objects = []
objects.append( threed.sphere([1,1,1], 0.5,color='#99FFFF') )
objects.append( threed.sphere([2,2,2], 0.5,color='#FFFF99') )
objects.append( threed.cylinder([1,1,1], [2,2,2], 0.2, color='#99FF99') )
p3j.Scene(children=[*objects])

Scene(children=(Mesh(geometry=SphereBufferGeometry(heightSegments=24, widthSegments=24), material=MeshPhongMat…

For serious work we need more control, so we will use `threed.render`.  This is actually a wrapper around `pythreejs.Renderer`:

In [41]:
threed.render (objects, imageSize=[640,240])

Renderer(camera=PerspectiveCamera(aspect=2.6666666666666665, children=(DirectionalLight(color='#FFFFFF', posit…

## Example 2: Draw the Atomium

As before, create a list called `objects`, whose members are instances of `pythreejs.Mesh`, and pass it to `threed.render`:

In [42]:
import numpy as np
objects = []
objects.append ( threed.cylinder ([0,0,0], [4,0,0], radius=.2, color='#FF0000') ) # red x axis
objects.append ( threed.cylinder ([0,0,0], [0,4,0], radius=.2, color='#00FF00') ) # green y axis
objects.append ( threed.cylinder ([0,0,0], [0,0,4], radius=.2, color='#0000FF') ) # blue z axis
points = np.array([[0,0,0],[2,0,0],[0,2,0],[0,0,2],[0,2,2],[2,2,0],[2,0,2],[2,2,2],[1,1,1]])
for point in points:
  objects.append ( threed.sphere (point, radius=np.sqrt(.75), color='#999999') )  

In [43]:
threed.render (objects)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

For more control, set the camera orientation and other parameters:

In [46]:
threed.render (objects, 
               camFov=np.radians(120),   # wide field of view leads to considerable distortion
               camOri=threed.chooseCameraOrientation (e=np.radians(20), a=np.radians(60)))

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [8]:
Jld = np.diag([5,4,1])
objects = []
rnd = np.array([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[0,1,1],[1,1,0],[1,0,1],[1,1,1]]) @ Jld # bounding box corners
#======== Add spheres at corners of cuboid
for rd in rnd:
  objects.append ( threed.sphere (rd, radius=.5, color='#9999FF') ) # mark each corner
#======== Add cylinders along edges of cuboid
for u in [0,1]:
  for v in [0,1]:
    for w in [0,1,2]:
      rdA = np.roll ([u,v,0], w) @ Jld
      rdB = np.roll ([u,v,1], w) @ Jld
      objects.append ( threed.cylinder (rdA, rdB, radius=.2, color='#99FF99') )
threed.render (objects, zoomOut=1)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

## Example 3: Text

In [27]:
import threed; import importlib; importlib.reload (threed);  # Developer use

In [68]:
objects = []
objects.append ( threed.cylinder ([0,0,0], [4,0,0], radius=.2, color='#FF0000') ) # red x axis
objects.append ( threed.cylinder ([0,0,0], [0,4,0], radius=.2, color='#00FF00') ) # green y axis
objects.append ( threed.cylinder ([0,0,0], [0,0,4], radius=.2, color='#0000FF') ) # blue z axis
points = np.array([[0,0,0],[2,0,0],[0,2,0],[0,0,2],[0,2,2],[2,2,0],[2,0,2],[2,2,2],[1,1,1]])
for point in points:
  objects.append ( threed.sphere (point, radius=np.sqrt(.75), color='#999999') )  
objects.append ( threed.billboard ('Atomium', position=[1.0, -1.2,0]) )  # add text
objects.append ( threed.billboard ('Fe', position=[2.0, 2.0, 2.9]) )     # add more text
threed.render (objects, elev=np.radians(60), azim=np.radians(270))       # almost a bird's eye view 

Setting camOri according to elev and azim


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

## Stock

For a perspective camera, we are given the vertical angular field of view (FoV) $\beta$, which is typically 50 degrees, and the aspect ratio of the camera view $Y/X$.
Let $q = \cot\frac{\beta}{2}$ and $p = \frac{X}{Y} \cot\frac{\beta}{2}$.

Now, take the bounding box corners $r_{nd}$ and perform the following.
1. Translate so that view center is at origin.
2. Rotate about origin so that viewing direction is (0,0,-1).
3. For each coordinate $(x_d,y_d,z_d)$, calculate $z_n + \max (p \left|x_n\right|, q \left|y_n\right|)$
4. Set the view distance to be $z = \max_n z_n$.  That is, put the camera position at CoM - ...

In [ ]:
# np.set_printoptions (precision=2,floatmode='fixed',suppress=True)

In [ ]:
# for pointA,pointB in itertools.combinations (points, 2):
#   if np.linalg.norm(pointB - pointA) < 5.01:
#     objects.append ( threed.cylinder (pointA, pointB, radius=.2, color='#99FF99') )